# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
os.chdir('C:\Users\chads\Desktop')

In [3]:
data = pd.io.stata.read_stata('us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
black_a = sum(data[data.race=='b'].call)
print(black_a)

157.0


As variables, black and white, are categorical we'll use hypothesis test for comparing two proportions. In order to meet CLT conditions for two proportions: 

a) Independence: Sample observations must be independent

   1) random sample assignment
   
   2) if sampling without replacement, n is less than 10% of the population
   
b) Sample size / skew:

   There should be at least 10 successes and 10 failures in the sample
   
   np>=10 and n(1-p)>=10
   
If these conditions are met, then may apply CLT

In [96]:
data.head(1)

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,


In [93]:
black = data[data.race == 'b']
black.head(1)

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,


In [23]:
# Number of black applicants
black_n = len(black)
black_n

2435

In [62]:
# Number of calls to black applicants
black_call = len(black[black.call==1.0])
black_call

157

In [38]:
# Number of white applicants
white_n = len(white)
white_n

2435

In [63]:
# Count how many whites got calls
white = data[data.race == 'w']
white_call = len(white[white.call==1.0])
white_call

235

In [65]:
# Total calls
calls = len(data[data.call==1.0])
calls

392

In [69]:
# Sample proportions:
w_sample_pro = white_call*1.0 / white_n
b_sample_pro = black_call*1.0 / black_n

print('Percentage of white call backs: ', w_sample_pro)
print('Percentage of black call backs: ', b_sample_pro)

('Percentage of white call backs: ', 0.09650924024640657)
('Percentage of black call backs: ', 0.06447638603696099)


Get the population pooled proportion

In [99]:
ppool= round((white_call + black_call)*1.0 / (white_n + black_n),2)
ppool

0.08

Check condition that there are 10 successes and 10 failures in the samples. Appears all are > 10. So all conditions of CLT are met for this two-sample proportion test

In [101]:
# check if n1p_pool >=10, n1(1-p_pool) >=10:
np_w= white_n*ppool
n1p_w= white_n*(1-ppool)

# and for n2p_pool >=10, n2(1-p_pool) >=10:
np_b= black_n*ppool
n1p_b= black_n*(1-ppool)

print np_w,n1p_w, np_b,n1p_b

194.8 2240.2 194.8 2240.2


Ho: proportion of whites = proportion of blacks

Ha: proportion of whites != proportion of blacks

In [102]:
SE_diff=round((((ppool*(1-ppool))/black_n)+((ppool*(1-ppool))/white_n))**0.5,2)
SE_diff

0.01

Get z-score and p_value..

In [78]:
from scipy.stats import norm, chi2_contingency

In [91]:
# Get z-score and p-value
arr = np.array([[white_call, white_n-white_call], [black_call, black_n-black_call]])
chi2, p_value, dof, exp = chi2_contingency(arr, correction=False)
print('     Z-score            p_value')
print(['{:.12f}'.format(d) for d in (chi2**0.5, p_value)])

     Z-score            p_value
['4.108412152434', '0.000039838868']


In [105]:
Z = float(4.108412152434)

Margin of error calculation

In [106]:
ME= round(Z * SE_diff,2)
print(ME)

0.04


Get confidence interval 

In [109]:
max_CI= (w_sample_pro - b_sample_pro)+ME
min_CI= (w_sample_pro - b_sample_pro)-ME

print " Confidence Interval: (",min_CI, ",", max_CI,")"

 Confidence Interval: ( -0.00796714579055 , 0.0720328542094 )


With an extremely low p-value of nearly zero, 0.00003, reject Ho that proportion of white calls equals the proportion of black calls, therefore could conclude racial discrimination is present in the U.S. job market